# Description

**TODO**

# Modules

In [1]:
library(reticulate)
pd <- import("pandas")

# Settings

In [2]:
# reference panel
REFERENCE_PANEL <- "GTEX_V8"
# REFERENCE_PANEL = "1000G"

# prediction models
## mashr
EQTL_MODEL <- "MASHR"

chromosome <- NULL

In [3]:
# Parameters
chromosome = 2
REFERENCE_PANEL = "1000G"
EQTL_MODEL = "MASHR"


In [4]:
paste0("Using reference panel: ", REFERENCE_PANEL)

[1] "Using reference panel: 1000G"

In [5]:
paste0("Using eQTL model: ", EQTL_MODEL)

[1] "Using eQTL model: MASHR"

In [6]:
# chromosome must be provided as parameter
stopifnot(!is.null(chromosome))

# Paths

In [7]:
GENE_CORRS_DIR <- Sys.getenv("PHENOPLIER_PHENOMEXCAN_LD_BLOCKS_GENE_CORRS_DIR")
IRdisplay::display(GENE_CORRS_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs"

In [8]:
INPUT_DIR <- file.path(GENE_CORRS_DIR, tolower(REFERENCE_PANEL), tolower(EQTL_MODEL), "by_chr")
IRdisplay::display(INPUT_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/1000g/mashr/by_chr"

In [9]:
INPUT_FILE <- file.path(INPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(INPUT_FILE)
stopifnot(file.exists(INPUT_FILE))

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/1000g/mashr/by_chr/gene_corrs-chr2.pkl"

In [10]:
OUTPUT_DIR <- file.path(INPUT_DIR, "corrected_positive_definite")
IRdisplay::display(OUTPUT_DIR)
dir.create(OUTPUT_DIR, recursive = TRUE, showWarnings = FALSE)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/1000g/mashr/by_chr/corrected_positive_definite"

In [11]:
OUTPUT_FILE <- file.path(OUTPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(OUTPUT_FILE)
if (file.exists(OUTPUT_FILE)) {
  IRdisplay::display("Output file exists, it will be overwritten")
}

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/1000g/mashr/by_chr/corrected_positive_definite/gene_corrs-chr2.pkl"

# Functions

In [12]:
# taken from https://www.r-bloggers.com/2013/08/correcting-a-pseudo-correlation-matrix-to-be-positive-semidefinite/
# TODO: add documentation
CorrectCM <- function(CM, p = 0) {
  n <- dim(var(CM))[1L]
  E <- eigen(CM)
  CM1 <- E$vectors %*% tcrossprod(diag(pmax(E$values, p), n), E$vectors)
  Balance <- diag(1 / sqrt(diag(CM1)))
  CM2 <- Balance %*% CM1 %*% Balance
  return(CM2)
}

In [13]:
is_positive_definite <- function(data) {
  # eigenvalues <- eigen(data)$values
  # nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]

  cholStatus <- try(u <- chol(data), silent = FALSE)
  cholError <- ifelse(class(cholStatus) == "try-error", TRUE, FALSE)

  if (cholError) {
    return(FALSE)
  } else {
    return(TRUE)
  }
}

# Load data

In [14]:
gene_corrs <- pd$read_pickle(INPUT_FILE)

In [15]:
dim(gene_corrs)

[1] 445 445

In [16]:
head(gene_corrs[1:10, 1:10])

,ENSG00000143727,ENSG00000115705,ENSG00000130508,ENSG00000182551,ENSG00000171865,ENSG00000171863,ENSG00000134326,ENSG00000134321,ENSG00000115738,ENSG00000134313
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000143727,1.000000000,0.010931472,0.009968052,0.00988758,0.01396987,0.009624337,0.000000000,0.000000000,0,0
ENSG00000115705,0.010931472,1.000000000,0.032124187,0.01425489,0.01534491,0.005077138,0.000000000,0.000000000,0,0
ENSG00000130508,0.009968052,0.032124187,1.000000000,0.01149480,0.01807399,0.009865334,0.000000000,0.000000000,0,0
ENSG00000182551,0.009887580,0.014254887,0.011494798,1.00000000,0.13845306,0.232627265,0.007882798,0.005280475,0,0
ENSG00000171865,0.013969871,0.015344908,0.018073988,0.13845306,1.00000000,0.132968656,0.015524953,0.009595429,0,0
ENSG00000171863,0.009624337,0.005077138,0.009865334,0.23262727,0.13296866,1.000000000,0.008745160,0.008610076,0,0


# Check positive definiteness

In [17]:
orig_matrix_is_positive_definite <- is_positive_definite(gene_corrs)

Error in chol.default(data) : 
  the leading minor of order 65 is not positive definite


In [18]:
if (orig_matrix_is_positive_definite) {
  IRdisplay::display("Matrix is already positive definite!")
  file.copy(INPUT_FILE, OUTPUT_FILE)
} else {
  IRdisplay::display("We need to correct the data and make the matrix positive definite")

  eigenvalues <- eigen(gene_corrs)$values
  nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
  IRdisplay::display(length(nonpositive_eigenvalues))
  IRdisplay::display(nonpositive_eigenvalues)

  # Make matrix positive definite
  gene_corrs_corrected <- CorrectCM(gene_corrs, 1e-10)
  dimnames(gene_corrs_corrected)[[1]] <- rownames(gene_corrs)
  dimnames(gene_corrs_corrected)[[2]] <- colnames(gene_corrs)
  gene_corrs_corrected <- as.data.frame(gene_corrs_corrected)

  # check if new matrix is really positive definite
  matrix_is_positive_definite <- is_positive_definite(gene_corrs_corrected)
  if (matrix_is_positive_definite) {
    IRdisplay::display("It worked!")
    IRdisplay::display(head(gene_corrs[1:10, 1:10]))
    IRdisplay::display(head(gene_corrs_corrected[1:10, 1:10]))
    py_save_object(gene_corrs_corrected, OUTPUT_FILE)
  } else {
    eigenvalues <- eigen(gene_corrs_corrected)$values
    nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
    IRdisplay::display(length(nonpositive_eigenvalues))
    IRdisplay::display(nonpositive_eigenvalues)

    stop("Method failed to adjust matrix")
  }
}

[1] "We need to correct the data and make the matrix positive definite"

[1] 1

[1] -1.875706e-05

[1] "It worked!"

,ENSG00000143727,ENSG00000115705,ENSG00000130508,ENSG00000182551,ENSG00000171865,ENSG00000171863,ENSG00000134326,ENSG00000134321,ENSG00000115738,ENSG00000134313
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000143727,1.000000000,0.010931472,0.009968052,0.00988758,0.01396987,0.009624337,0.000000000,0.000000000,0,0
ENSG00000115705,0.010931472,1.000000000,0.032124187,0.01425489,0.01534491,0.005077138,0.000000000,0.000000000,0,0
ENSG00000130508,0.009968052,0.032124187,1.000000000,0.01149480,0.01807399,0.009865334,0.000000000,0.000000000,0,0
ENSG00000182551,0.009887580,0.014254887,0.011494798,1.00000000,0.13845306,0.232627265,0.007882798,0.005280475,0,0
ENSG00000171865,0.013969871,0.015344908,0.018073988,0.13845306,1.00000000,0.132968656,0.015524953,0.009595429,0,0
ENSG00000171863,0.009624337,0.005077138,0.009865334,0.23262727,0.13296866,1.000000000,0.008745160,0.008610076,0,0


,ENSG00000143727,ENSG00000115705,ENSG00000130508,ENSG00000182551,ENSG00000171865,ENSG00000171863,ENSG00000134326,ENSG00000134321,ENSG00000115738,ENSG00000134313
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000143727,1.000000000,0.010931472,0.009968052,0.00988758,0.01396987,0.009624337,2.558717e-17,2.190088e-17,-1.212612e-17,2.375080e-18
ENSG00000115705,0.010931472,1.000000000,0.032124187,0.01425489,0.01534491,0.005077138,-5.789640e-17,-8.290894e-16,4.648144e-16,5.699448e-16
ENSG00000130508,0.009968052,0.032124187,1.000000000,0.01149480,0.01807399,0.009865334,-2.001437e-16,6.481361e-16,-5.114249e-16,-6.929678e-16
ENSG00000182551,0.009887580,0.014254887,0.011494798,1.00000000,0.13845306,0.232627265,7.882798e-03,5.280475e-03,-2.001437e-16,6.288373e-18
ENSG00000171865,0.013969871,0.015344908,0.018073988,0.13845306,1.00000000,0.132968656,1.552495e-02,9.595429e-03,-1.125402e-16,-1.373684e-16
ENSG00000171863,0.009624337,0.005077138,0.009865334,0.23262727,0.13296866,1.000000000,8.745160e-03,8.610076e-03,-9.074772e-17,-1.486441e-16


Both matrices should "look" similar. We are not interested in perfectly accurate correlation values (they are already inaccurate).